In [1]:
!pip install langchain_community pypdf openai langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
from langchain_community.document_loaders import PyPDFLoader
import os
from google.colab import userdata
from langchain_openai import OpenAIEmbeddings


In [71]:
file_path = "/content/Think-And-Grow-Rich_2011-06.pdf"
loader = PyPDFLoader(file_path)

In [72]:
docs = loader.load()

In [73]:
docs[0]

Document(metadata={'producer': 'OpenOffice.org 3.2', 'creator': 'Writer', 'creationdate': '2011-06-06T18:53:08+01:00', 'title': 'Think and Grow Rich by Napoleon Hill', 'author': 'Brod', 'subject': 'Free Digital Download PDF eBook Edition', 'keywords': 'free, ebook, think and grow rich, make money online, home based business, opportunity', 'source': '/content/Think-And-Grow-Rich_2011-06.pdf', 'total_pages': 253, 'page': 0, 'page_label': '1'}, page_content='\uf09a  THINK & GROW RICH  \uf09b \nFree Digital Download PDF eBook Edition \nRe-published by: \n\uf09a  www.think-and-grow-rich-ebook.com \uf09b \ni')

In [75]:
docs[0].metadata

{'producer': 'OpenOffice.org 3.2',
 'creator': 'Writer',
 'creationdate': '2011-06-06T18:53:08+01:00',
 'title': 'Think and Grow Rich by Napoleon Hill',
 'author': 'Brod',
 'subject': 'Free Digital Download PDF eBook Edition',
 'keywords': 'free, ebook, think and grow rich, make money online, home based business, opportunity',
 'source': '/content/Think-And-Grow-Rich_2011-06.pdf',
 'total_pages': 253,
 'page': 0,
 'page_label': '1'}

In [76]:
docs[0].page_content

'\uf09a  THINK & GROW RICH  \uf09b \nFree Digital Download PDF eBook Edition \nRe-published by: \n\uf09a  www.think-and-grow-rich-ebook.com \uf09b \ni'

In [74]:
len(docs)

253

In [39]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [40]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [41]:
vector_store.add_documents(docs)

['3bb2d512-9a4e-4a85-9e51-4cc515aad568',
 '63ebdf0c-d041-42b0-8094-492ef6279d18',
 '9a50617e-d347-42d8-9075-9894646e16b2',
 'c7e85dbf-def2-402c-95c3-28232f2511a2',
 '0a37e55c-eb74-49e1-934b-3d1c03f64f42',
 'b5cdcd24-8025-4331-b392-8e5c0da59e5a',
 'd8a39732-2842-4071-b39c-345cde6f64a4',
 '8a3058fa-96db-463e-84dc-e10131cd4598',
 '4eddda54-2224-4c8b-b186-af15792bc78b',
 '5bcc0218-de00-4dc8-8330-1c5cbfc00c2f',
 'adf1bca4-4d17-4b68-b377-5396a95d0940',
 '38037d54-6196-479a-93a7-e0308eba87e8',
 '04abffdf-f6f5-47a9-977a-e8f078598bb3',
 '5077e6e9-4d8d-4bee-971e-39a8a0547ea5',
 '112dd7b8-d7b4-49af-ba2f-792996f32cb2',
 'dfa8289c-1e96-4e23-9519-01788f1c6c57',
 'aa186e1f-68ce-4306-aaf1-b7e3bea71731',
 '0562b22e-6650-42b3-a89d-22cbd57565c6',
 '770c1a50-1071-4157-b402-d0655786218c',
 'aca4ffdf-a00e-49db-87e4-840c93055bf0',
 '7ea98afe-5e5a-4100-b3f7-41b08466ae77',
 'a6d76492-7cf7-48ac-ad7c-5ac0c6a6acc5',
 'a8aa5d21-7f88-4fa1-9132-a50cdedb15a8',
 '65563b94-b8c6-4852-aa40-7338ade81c47',
 '46b2e954-86c5-

In [54]:
retriever = vector_store.as_retriever(search_kwargs = {"k":2})

In [55]:
retrieved_docs = retriever.invoke("what did the president say about ketanji brown jackson?")

In [61]:
from openai import OpenAI

client = OpenAI(api_key = userdata.get('OPENAI_API_KEY'))

I'm called ChatGPT. How can I assist you today?


In [70]:
SYSTEM_PROMPT = "You are a helpful assitant who answers questions based on a document uploaded. User ask a question and a context docunments is provided and you answer based on that. If the context does not answer user's question, you simply tell user that it does not provide the answer"
messages = [{"role" : "system", "content" : SYSTEM_PROMPT}]

def prepare_user_message(query, context_docs):
  contexr_text = "\n\n".join([d.page_content for d in context_docs])
  prompt = "Answer the following user query concisely under 1000 words: {0}.\n\n Based on the below context\n\n: {1}".format(query, contexr_text)
  return {"role":"user", "content" : prompt}

while True:
  query = input("What do you want to ask? ")
  # query = "How can someone become rich accoring to this book?"
  ret_docs = retriever.invoke(query)
  messages.append(prepare_user_message(query, ret_docs))
  response = client.responses.create(
      model = "gpt-4o-mini",
      input = messages,
      max_output_tokens = 1000
  )
  print("Here is what I think: ", response.output_text)

What do you want to ask? How are you doing?
Here is what I think:  The context provided does not directly address the question "How are you doing?" It poses various introspective questions related to one's mental state, personal satisfaction, and reactions to life’s challenges. If you have specific concerns about your well-being or feelings, I can help guide a discussion based on the principles outlined in the text.
What do you want to ask? Who is the author of this book?
Here is what I think:  The author of "Think and Grow Rich" is Napoleon Hill. He was inspired to write this book after being introduced to the wealth-building philosophies of Andrew Carnegie, who provided him with the foundational principles that the book discusses. Hill spent over twenty years researching and compiling the experiences of more than 500 wealthy individuals to distill the secrets to financial success.
What do you want to ask? what is he talking about for fear?
Here is what I think:  In "Think and Grow Ri

KeyboardInterrupt: Interrupted by user